# Run calibration 1.4.2

Here we test running the volcanic forcing time series annually with calibration v1.4.2, developed specifically for this volcanic forcing time series.

Taking a median across parameters is unsatisfactory, and so these projections are not the best.

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
scenarios = ["ssp245"]

In [ ]:
df_solar = pd.read_csv(
    "../data/solar-forcing/forcing_yearly_1750-2300.csv", index_col=0
)
df_volcanic_hist = pd.read_csv(
    "../data/volcanic-forcing/yearly/forcing_yearly_1750-2014.csv", index_col=0
)

In [ ]:
volcanic_future = np.zeros((86, 1000))
for config in range(1000):
    df_yearly_scen = pd.read_csv(
        f"../data/volcanic-forcing/yearly/forcing_yearly_scen{config+1}.csv", index_col=0
    )
    volcanic_future[:, config] = df_yearly_scen['Global yearly mean TOA ERF (W/m2)'] - df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()

In [ ]:
pl.plot(volcanic_future);

In [ ]:
#df_monthly_scen1

In [ ]:
pl.plot(df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'])
pl.plot(df_yearly_scen['Global yearly mean TOA ERF (W/m2)'])

In [ ]:
df_solar

In [ ]:
solar_forcing = np.zeros(352)
volcanic_forcing = np.zeros(352)
solar_forcing = df_solar["solar_erf"].loc[1750:2101].values
## remove historical mean
#volcanic_forcing[1:12*265+1] = df_volcanic_hist['Global monthly mean TOA ERF (W/m2)'].values - df_volcanic_hist['Global monthly mean TOA ERF (W/m2)'].values.mean()
#volcanic_forcing[12*265+1:] = df_monthly_scen1['Global monthly mean TOA ERF (W/m2)'].values - df_volcanic_hist['Global monthly mean TOA ERF (W/m2)'].values.mean()

In [ ]:
df_methane = pd.read_csv(
    "../data/fair2.1-parameters/calibration-1.4.2/CH4_lifetime.csv",
    index_col=0,
)
df_configs = pd.read_csv(
    "../data/fair2.1-parameters/calibration-1.4.2/calibrated_constrained_parameters.csv",
    index_col=0,
)
valid_all = df_configs.index

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2101, 1)
f.define_scenarios(scenarios)
#f.define_configs(['median'])
f.define_configs(list(range(1000)))
species, properties = read_properties()
species.remove("Halon-1202")
species.remove("NOx aviation")
species.remove("Contrails")
f.define_species(species, properties)
f.allocate()

In [ ]:
da_emissions = xr.load_dataarray(
    "../data/emissions/ssp_emissions_1750-2500.nc"
)

In [ ]:
da = da_emissions.loc[dict(config="unspecified", scenario="ssp245")][:351, ...]
fe = da.expand_dims(dim=["scenario", "config"], axis=(1, 2))
f.emissions = fe.drop_vars("config") * np.ones((1, 1, 1000, 1))

In [ ]:
f.timebounds.shape

In [ ]:
# solar and volcanic forcing
for iconf in range(1000):
    f.forcing.loc[
        dict(
            specie='Volcanic',
            timebounds=1750,
            config=iconf,
            scenario='ssp245'
        )
    ] = - df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
    f.forcing.loc[
        dict(
            specie='Volcanic',
            timebounds=np.arange(1751, 2016),
            config=iconf,
            scenario='ssp245'
        )
    ] = df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].values - df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
    f.forcing.loc[
        dict(
            specie='Volcanic',
            timebounds=np.arange(2016, 2102),
            config=iconf,
            scenario='ssp245'
        )
    ] = volcanic_future[:, iconf]

In [ ]:
fill(
    f.forcing,
    solar_forcing[:, None, None],# * df_configs["fscale_solar_amplitude"].values.squeeze(),
    specie="Solar",
)

# climate response
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs.loc[:, "clim_c1":"clim_c3"].median().values,
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].median().values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    f.climate_configs["deep_ocean_efficacy"],
    df_configs["clim_epsilon"].median(),
)
fill(
    f.climate_configs["gamma_autocorrelation"],
    df_configs["clim_gamma"].median(),
)
fill(f.climate_configs["sigma_eta"], df_configs["clim_sigma_eta"].median())
fill(f.climate_configs["sigma_xi"], df_configs["clim_sigma_xi"].median())
#fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
#fill(f.climate_configs["stochastic_run"], True)
#fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["clim_F_4xCO2"].median())

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs["cc_r0"].median(), specie="CO2")
fill(
    f.species_configs["iirf_airborne"],
    df_configs["cc_rA"].median(),
    specie="CO2",
)
fill(
    f.species_configs["iirf_uptake"], df_configs["cc_rU"].median(), specie="CO2"
)
fill(
    f.species_configs["iirf_temperature"],
    df_configs["cc_rT"].median(),
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs["aci_beta"].median())
fill(
    f.species_configs["aci_shape"],
    df_configs["aci_shape_so2"].median(),
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"],
    df_configs["aci_shape_bc"].median(),
    specie="BC",
)
fill(
    f.species_configs["aci_shape"],
    df_configs["aci_shape_oc"].median(),
    specie="OC",
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs[f"ari_{specie}"].median(),
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs[f"fscale_{specie}"].median(),
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs["fscale_minorGHG"].median(),
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs[f"o3_{specie}"].median(),
        specie=specie,
    )

# tune down volcanic efficacy
fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs["cc_co2_concentration_1750"].median(),
    specie="CO2",
)

# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.figure(figsize=(4, 4))
pl.plot(f.timebounds, f.temperature.sel(layer=0, scenario='ssp245') - f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901, 1/12), method="nearest").mean());
pl.xlim(1850, 2100)
pl.ylim(-0.5, 2.8)
os.makedirs('../plots', exist_ok=True)
pl.savefig('../plots/stochastic_volcanoes_all.png')

In [ ]:
pl.figure(figsize=(4, 4))
pl.fill_between(
    f.timebounds, 
    (
        f.temperature.sel(layer=0, scenario='ssp245') - 
        f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901), method="nearest").mean()
    ).quantile(0.00, dim='config'),
    (
        f.temperature.sel(layer=0, scenario='ssp245') - 
        f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901), method="nearest").mean()
    ).quantile(1.00, dim='config'),
    color='0.85'
);
pl.fill_between(
    f.timebounds, 
    (
        f.temperature.sel(layer=0, scenario='ssp245') - 
        f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901), method="nearest").mean()
    ).quantile(0.05, dim='config'),
    (
        f.temperature.sel(layer=0, scenario='ssp245') - 
        f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901), method="nearest").mean()
    ).quantile(0.95, dim='config'),
    color='0.7'
);
pl.fill_between(
    f.timebounds, 
    (
        f.temperature.sel(layer=0, scenario='ssp245') - 
        f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901), method="nearest").mean()
    ).quantile(0.16, dim='config'),
    (
        f.temperature.sel(layer=0, scenario='ssp245') - 
        f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901), method="nearest").mean()
    ).quantile(0.84, dim='config'),
    color='0.55'
);
pl.plot(
    f.timebounds, 
    (
        f.temperature.sel(layer=0, scenario='ssp245') - 
        f.temperature.sel(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901), method="nearest").mean()
    ).median(dim='config'),
    color='k'
);
pl.xlim(1850, 2100)
pl.ylim(-0.5, 2.8)

os.makedirs('../plots', exist_ok=True)
pl.savefig('../plots/stochastic_volcanoes.png')

In [ ]:
pl.figure(figsize=(4, 4))
for layer in range(1):
    pl.plot(
        f.timebounds, 
        (
            f.temperature.sel(layer=layer, scenario='ssp245', config=1) - 
            f.temperature.sel(layer=layer, scenario='ssp245', config=1, timebounds=np.arange(1850, 1901), method="nearest").mean()
        )
    );
    pl.xlim(1850, 2100)
pl.ylim(-0.5, 2.8)
os.makedirs('../plots', exist_ok=True)
pl.savefig('../plots/ensemble_member_1.png')